In [1]:
import NeighborhoodMarketExpansionModel as ngb
import expertForecastFunctionsCopy1 as ff
import regularModelFunctions as rmf
import visitorFunctions as vf
from regression_function import *
from financials_clean_year_assign import *
from improved_find_data_start_row import *

import numpy as np


In [2]:
walmartPath="/Users/myself/Desktop/Walmart USA Searching for Growth/"

financial_data=improved_load_excel_with_autodetect("/Users/myself/Desktop/Walmart USA Searching for Growth/walmartHistoricalFinancials.xlsx")
financial_data=clean_cashflow_data(financial_data, 3)
financial_data=update_column_names(financial_data, base_year=2021,index_name = 'Financial Metric')

cashflow_data=pd.read_excel(walmartPath+"walmartCashFlow.xlsx",skiprows=15)
cashflow_data=clean_cashflow_data(cashflow_data, 10)
cashflow_data=update_column_names(cashflow_data, base_year=2022,index_name = 'Cash Metric')
cashflow_pivot=pivot_year_columns_to_rows(cashflow_data,'Cash Metric')
cashflow_pivot=cashflow_pivot/1000

statement_pivot=pivot_year_columns_to_rows(financial_data)
statement_pivot=statement_pivot/1000

store_count=pd.read_excel(walmartPath+"walmartCashFlow.xlsx", "Yearly Store Count by Type",skiprows=2)
store_count=clean_cashflow_data(store_count, 10)
store_pivot=pivot_year_columns_to_rows(store_count,'Year')
store_pivot.rename(columns={"Total":"Total Store"},inplace=True)


distribution_count=pd.read_excel(walmartPath+"walmartCashFlow.xlsx", "Yearly DC", skiprows=1)
distribution_count.rename(columns={"Total":"Total DC","Total.1":"Total SqFt"},inplace=True)
dist_pivot=distribution_count.pivot_table(index='Year')
dist_pivot.index=dist_pivot.index.astype(str)
dist_pivot['Total SqFt Thousands'] = dist_pivot['Total SqFt']/1000


merge_data_left = pd.merge(cashflow_pivot, store_pivot, on='Year', how='left')
merge_data_left = pd.merge(merge_data_left, dist_pivot ,on='Year',how='left')
merge_data_left = pd.merge(merge_data_left, statement_pivot ,on='Year',how='left')



merge_left_numeric = merge_data_left.apply(pd.to_numeric, errors='coerce')
merge_left_numeric.index = pd.to_datetime(merge_left_numeric.index, format='%Y', errors='coerce')
merge_left_numeric.interpolate(method='linear', fill_value="extrapolate", limit_direction="both",inplace=True)
merge_left_numeric.isna().any(axis='columns')


model1= perform_ols_regression(merge_left_numeric,['Total SqFt Thousands'], 'Capital Expenditures')
model2= perform_ols_regression(merge_left_numeric,['Discount stores','Supercenters','Neighborhood markets','Capital Expenditures'], 'Total SqFt Thousands')



In [3]:
prob_matrix = np.array([
    [0.999, 0.989, 0.966, 0.906, 0.717, 0.496, 0.236],
    [0.999, 0.979, 0.941, 0.849, 0.610, 0.387, 0.172],
    [0.997, 0.962, 0.899, 0.767, 0.490, 0.288, 0.123],
    [0.995, 0.933, 0.834, 0.659, 0.372, 0.206, 0.086],
    [0.989, 0.883, 0.739, 0.531, 0.268, 0.142, 0.060],
    [0.978, 0.803, 0.615, 0.398, 0.184, 0.096, 0.041]
])


distances = np.array([0, 1, 2, 3, 4, 5])  # in miles
densities = np.array([1, 5, 10, 20, 50, 100, 250])  # thousands of people within a 5-mile radius

total_weekly_visits = 167*1e6
total_annual_visits = 167*1e6 * 52
total_net_sales = 393247*1e6 
avg_spend = (393247*1e6) / (8684*1e6)
avg_spending_per_visitor = {'Supercenters': 35.0, 'Discount Stores': 28.5, 'Neighborhood Markets': 32.0}

# Store-specific data with density eligibility
store_data = {
    'Supercenters': {'locations': 3573, 'avg_sqft': 178000, 'density_eligibility': np.array([1, .8, .8, .8, 0.3, 0, 0])},
    'Discount Stores': {'locations': 370, 'avg_sqft': 105000, 'density_eligibility': np.array([0, .2, .2, .2, 0, 0, 0])},
    'Neighborhood Markets': {'locations': 799, 'avg_sqft': 42000, 'density_eligibility': np.array([0, 0, 0, 0, 0.7, 1, 1])}
}

market_conditions={'community_opposition': 0.5, 'cannibalization_rate': 0.01, 'ecommerce_growth_factor': 0.03}

In [4]:


forecast_years = forecast_years = ['2022F','2023F', '2024F', '2025F', '2026F']
financial_data = ff.handle_missing_values(financial_data, model_cols=None)
financial_forecast = ff.linear_regression_forecast(financial_data, forecast_years, transform_if_needed=True)

curr_finances = financial_data['2021']

In [5]:
model = ngb.NeighborhoodMarketExpansionModel(
        curr_finances = curr_finances,
        store_data = store_data,
        prob_matrix = prob_matrix,
        distances = distances,
        densities = densities,
        avg_spending_per_visitor = avg_spending_per_visitor,
        financial_forecast = financial_forecast,
        regression_results = model2,
        market_conditions = market_conditions,  # Correctly initializing the market_conditions
        visitor_estimates = None)

estimated_visitors, visitors_from_each_density, visitors_by_store_type, visitor_results = vf.estimate_visitor_distribution_by_store_type(
total_visitors=total_annual_visits, prob_matrix=prob_matrix, distances=distances, densities=densities, store_data=store_data)
optimal_visitors = 1200000
max_stores = 50 


In [6]:
model.simulate_store_numbers(visitors_by_store_type, optimal_visitors, max_stores, 'Neighborhood Markets')

{1: (1236586.3879451547, 100),
 2: (1235042.5847142618, 100),
 3: (1233502.6313667377, 100),
 4: (1231966.5135194566, 100),
 5: (1230434.2168608503, 100),
 6: (1228905.7271504642, 100),
 7: (1227381.0302185158, 100),
 8: (1225860.1119654567, 100),
 9: (1224342.9583615393, 100),
 10: (1222829.5554463828, 100),
 11: (1221319.8893285478, 100),
 12: (1219813.9461851094, 100),
 13: (1218311.712261236, 100),
 14: (1216813.1738697707, 100),
 15: (1215318.3173908154, 100),
 16: (1213827.1292713175, 100),
 17: (1212339.5960246613, 100),
 18: (1210855.7042302615, 100),
 19: (1209375.4405331586, 100),
 20: (1207898.7916436186, 100),
 21: (1206425.7443367362, 100),
 22: (1204956.2854520385, 100),
 23: (1203490.4018930944, 100),
 24: (1202028.0806271248, 100),
 25: (1200569.3086846161, 100),
 26: (1199114.0731589377, 99.92617276324481)}

In [7]:
model.run_monte_carlo_simulations()

(-100384723.12190421, 66382.50042819873)

In [ ]:
import unittest
class TestNeighborhoodMarketExpansionModel(unittest.TestCase):
    def setUp(self):
        # Setup that will be run before each test method
        self.curr_finances = financial_data
        self.store_data = {'NeighborhoodMarket': {'locations': 50, 'average_labor_cost': 30000, 'current_distribution_sqft': 50000}}
        self.prob_matrix = prob_matrix
        self.distances = distances
        self.densities = densities
        self.avg_spending_per_visitor = {'NeighborhoodMarket': 45.50}
        self.financial_forecast = {'Revenue Forecast': 1200000, 'Capital Expenditures Forecast': 200000}
        self.regression_results = {'sqft_per_store': 1500, 'cost_per_sqft': 10}
        self.market_conditions = {'community_opposition': 0.5, 'cannibalization_rate': 0.01, 'ecommerce_growth_factor': 0.03}

        self.model = NeighborhoodMarketExpansionModel(
            curr_finances=self.curr_finances,
            store_data=self.store_data,
            prob_matrix=self.prob_matrix,
            distances=self.distances,
            densities=self.densities,
            avg_spending_per_visitor=self.avg_spending_per_visitor,
            financial_forecast=self.financial_forecast,
            regression_results=self.regression_results,
            market_conditions=self.market_conditions
        )

    def test_simulate_store_numbers(self):
        # Test to ensure that store number simulation stops correctly
        visitors_by_store_type = {'NeighborhoodMarket': 1000}
        optimal_visitors = 100
        max_stores = 20
        results = self.model.simulate_store_numbers(visitors_by_store_type, optimal_visitors, max_stores, 'NeighborhoodMarket')
        self.assertEqual(len(results), 11)  # Expecting 11 because it should stop adding stores when profitability drops below optimal

    def test_estimate_new_distribution_sqft(self):
        # Test to ensure correct estimation of new distribution square footage
        num_new_stores = 10
        current_total_stores = 50
        expected_sqft = 1500 * (60) - 50000
        actual_sqft = self.model.estimate_new_distribution_sqft(num_new_stores, current_total_stores)
        self.assertEqual(actual_sqft, expected_sqft)

    def test_adjust_operational_costs(self):
        # Test adjustment of operational costs based on market conditions
        base_cost = 100000
        expected_cost = base_cost * (1 - 0.2 * 0.5)
        actual_cost = self.model.adjust_operational_costs(base_cost)
        self.assertAlmostEqual(actual_cost, expected_cost)


In [ ]:
unittest.main(argv=[''], exit=False)
